CELL 1 — Setup paths

In [ ]:
storage = "your-storage-name"
# Bronze paths
bronze_products = f"abfss://datalake@{storage}.dfs.core.windows.net/bronze/fakestore/products/"
bronze_users    = f"abfss://datalake@{storage}.dfs.core.windows.net/bronze/fakestore/users/"
bronze_stream   = f"abfss://datalake@{storage}.dfs.core.windows.net/bronze/fakestore/stream/"
# Silver output paths
silver_products = f"abfss://datalake@{storage}.dfs.core.windows.net/silver/fakestore/products/"
silver_users    = f"abfss://datalake@{storage}.dfs.core.windows.net/silver/fakestore/users/"
silver_orders   = f"abfss://datalake@{storage}.dfs.core.windows.net/silver/fakestore/orders/"


StatementMeta(spkecommerce, 8, 10, Finished, Available, Finished)

CELL 2 — Load Bronze Data 


In [10]:
df_prod_bronze = spark.read.parquet(bronze_products)
df_user_bronze = spark.read.parquet(bronze_users)
df_order_bronze = spark.read.parquet(bronze_stream)


StatementMeta(spkecommerce, 8, 11, Finished, Available, Finished)

CELL 3 — Clean Products (Silver)

In [11]:
from pyspark.sql.functions import col

df_prod_silver = (
    df_prod_bronze
    .dropDuplicates(["id"])               # remove duplicates
    .withColumnRenamed("id", "product_id")
)

df_prod_silver.write.mode("overwrite").parquet(silver_products)
print("Silver products created.")


StatementMeta(spkecommerce, 8, 12, Finished, Available, Finished)

Silver products created.


CELL 4 — Clean Users (Silver)

In [12]:
df_user_silver = (
    df_user_bronze
    .dropDuplicates(["id"])
    .withColumnRenamed("id", "user_id")
)

df_user_silver.write.mode("overwrite").parquet(silver_users)
print("Silver users created.")


StatementMeta(spkecommerce, 8, 13, Finished, Available, Finished)

Silver users created.



CELL 5 — Clean Orders (from Streaming)

In [13]:
from pyspark.sql.functions import to_timestamp

df_order_silver = (
    df_order_bronze
    .dropDuplicates(["cart_id", "product_id", "user_id", "ingest_ts"])
    .withColumn("order_ts", to_timestamp(col("cart_date")))
)

df_order_silver.write.mode("overwrite").parquet(silver_orders)
print("Silver orders created.")


StatementMeta(spkecommerce, 8, 14, Finished, Available, Finished)

Silver orders created.
